In [1]:
import pandas as pd

In [2]:
# Note: this data is normally available directly from Soundcast outputs
# but the 2018 RTP used outdated geographic lookups. The CSV files were re-generated and will be
# summarized here.

# VMT

In [3]:

df_vmt = pd.read_csv(r'L:\RTP_2022\final_runs\sc_rtp_2018_final\soundcast\outputs\agg\dash\person_vmt.csv')
df_person = pd.read_csv(r'L:\RTP_2022\final_runs\sc_rtp_2018_final\soundcast\outputs\agg\dash\person_geog.csv')

df_vmt['is_rgc'] = 0
df_vmt.loc[df_vmt['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1
df_person['is_rgc'] = 0
df_person.loc[df_person['hh_rgc'] != 'Not in RGC', 'is_rgc'] = 1

# Select only drivers (dorp = 1) and auto trips
df_vmt = df_vmt[df_vmt['mode'].isin(['SOV','HOV2','HOV3+']) & (df_vmt['dorp'] == 1)]

def vmt_per_person(df_vmt, df_person, geog):
    _df_vmt = df_vmt.groupby(geog).sum()[['travdist_wt']]
    _df_person = df_person.groupby(geog).sum()[['psexpfac']]

    df = _df_vmt.merge(_df_person, left_index=True, right_index=True)
    df.loc['Region',:] = df.sum(axis=0)
    df['VMT per Capita'] = df['travdist_wt']/df['psexpfac']
    
    return df

df = vmt_per_person(df_vmt, df_person, 'hh_rgc')
df

,travdist_wt,psexpfac,VMT per Capita
hh_rgc,,,
Auburn,1.685982e+04,1406.0,11.991336
Ballard-Interbay,4.105341e+03,391.0,10.499592
Bellevue,1.018351e+05,14209.0,7.166944
Bothell Canyon Park,9.393326e+03,466.0,20.157352
Bremerton,1.349682e+04,2556.0,5.280445
Burien,4.819448e+04,3449.0,13.973466
Cascade,2.128409e+04,1233.0,17.262038
Duwamish,1.208466e+04,1119.0,10.799515
Everett,5.723879e+04,6199.0,9.233553


In [4]:
df.to_clipboard()

In [5]:
df = vmt_per_person(df_vmt, df_person, 'hh_county')
df

,travdist_wt,psexpfac,VMT per Capita
hh_county,,,
King,3.415406e+07,2149234.0,15.891269
Kitsap,3.896457e+06,257159.0,15.151936
Pierce,1.509726e+07,851709.0,17.725848
Snohomish,1.575732e+07,795052.0,19.819238
Region,6.890510e+07,4053154.0,17.000366


In [6]:
df.to_clipboard()

# Mode Share

## To MICs

In [47]:
df_mode_tour = pd.read_csv(r'L:\RTP_2022\final_runs\sc_rtp_2018_final\soundcast\outputs\agg\dash\tour_rgc_dest.csv')
# Select all modes except for School Bus because we're dealing with work commuters
# There are some work trips where school bus is taken (high school workers) but never to MICs.
# However, regional mode shares for these trips do have school bus trips but it's best to exclude these for simplicity
df_mode_tour = df_mode_tour[df_mode_tour['tmodetp'] != 'School Bus']

In [48]:
rgc_tot = df_mode_tour[df_mode_tour['pdpurp'] == 'Work'].groupby('tour_d_rgc').sum()[['toexpfac']].reset_index()
rgc_mode = df_mode_tour[df_mode_tour['pdpurp'] == 'Work'].groupby(['tour_d_rgc','tmodetp']).sum()[['toexpfac']].reset_index()

df_work_rgc = rgc_mode.merge(rgc_tot, on='tour_d_rgc', how='left')
df_work_rgc['mode_share_trips_to_rgc'] = df_work_rgc['toexpfac_x']/df['toexpfac_y']

In [50]:
df_work_rgc[['tour_d_rgc','tmodetp','mode_share_trips_to_rgc']]

,tour_d_rgc,tmodetp,mode_share_trips_to_rgc
0,Auburn,Bike,0.011526
1,Auburn,HOV2,0.165477
2,Auburn,HOV3+,0.080132
3,Auburn,Park,0.025247
4,Auburn,SOV,0.641603
...,...,...,...
318,University Place,Park,0.014462
319,University Place,SOV,0.636638
320,University Place,TNC,0.008436
321,University Place,Transit,0.020187


In [51]:
pd.pivot_table(df_work_rgc,index='tour_d_rgc', columns='tmodetp', aggfunc='sum')[['mode_share_trips_to_rgc']].to_clipboard()

In [26]:
# NOTE: I have renamed TNC to Other since it includes other modes. I beleive this was also done in the RTP

In [46]:
# df_mode_tour.groupby('tmodetp').sum()[['toexpfac']]/df_mode_tour[['toexpfac']].sum()

## All MICs Combined

In [68]:
mic_list = ['Ballard-Interbay',
           'Duwamish',
           'Kent MIC',
           'North Tukwila',
           'Puget Sound Industrial Center- Bremerton',
           'Frederickson',
           'Port of Tacoma',
           'Sumner Pacific',
           'Cascade',
           'Paine Field / Boeing Everett']

all_mic_df = df_mode_tour[(df_mode_tour['pdpurp'] == 'Work') & (df_mode_tour['tour_d_rgc'].isin(mic_list))]

rgc_tot = all_mic_df.groupby('tmodetp').sum()[['toexpfac']].reset_index()
rgc_tot['all_mic_mode_share'] = rgc_tot['toexpfac']/rgc_tot['toexpfac'].sum()

rgc_tot.to_clipboard()
rgc_tot

,tmodetp,toexpfac,all_mic_mode_share
0,Bike,2024.0,0.011165
1,HOV2,31106.0,0.171585
2,HOV3+,15584.0,0.085964
3,Park,2452.0,0.013526
4,SOV,119830.0,0.661000
5,TNC,1555.0,0.008578
6,Transit,7407.0,0.040858
7,Walk,1328.0,0.007325


## Region

In [70]:

df = df_mode_tour[(df_mode_tour['pdpurp'] == 'Work')]

region_tot = df.groupby('tmodetp').sum()[['toexpfac']].reset_index()
region_tot['all_mic_mode_share'] = region_tot['toexpfac']/region_tot['toexpfac'].sum()

region_tot.to_clipboard()
region_tot

,tmodetp,toexpfac,all_mic_mode_share
0,Bike,20148.0,0.011792
1,HOV2,274395.0,0.160593
2,HOV3+,137766.0,0.080629
3,Park,39953.0,0.023383
4,SOV,1031617.0,0.603765
5,TNC,17648.0,0.010329
6,Transit,117401.0,0.068710
7,Walk,69713.0,0.040800


In [29]:
df

,toexpfac
tmodetp,
Bike,0.012106
HOV2,0.246604
HOV3+,0.218001
Park,0.006420
SOV,0.358486
TNC,0.016167
Transit,0.035242
Walk,0.106974
